# Plantaris Data - Data labeling system

The **data labeling process** is hard and tedious. In this notebook you are going to implement a system to simplify this process. 

The data from the sensors DHT22 (temperature and relative humidity) and soil moisture was collected and save it in a .CSV file.

### Import libraries 

In [24]:
import os
import csv
import random
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output

filename = "temp_hum_processed.csv"
df = None
row = None

if os.path.isfile(filename):
    df = pd.read_csv(filename, sep=";")
    print("DataFrame loaded")
else:
    print(f"ERROR: filename '{filename}' doesn't exist.")

if "category" not in df.columns:
    print("The file doesn't have a 'category' column, creating it")
    df["category"] = None

DataFrame loaded
The file doesn't have a 'category' column, creating it


### Widgets Creation

In [21]:
## Widgets
out = widgets.Output()

# Image Widget
w_image = widgets.Image(
        format='png',
        width=600,
        height=800,
    )

# Text for values
w_temperature = widgets.Text(
    description='Temperature:',
    disabled=True,
)

w_humidity = widgets.Text(
    description='Humidity:',
    disabled=True,
)

w_moisture = widgets.Text(
    description='Moisture:',
    disabled=True,
)

# Buttons Widgets
save_button = widgets.Button(
    description="Save",
    icon='check',
    button_style="info"
)
b1 = widgets.Button(
    description='Ok',
    button_style="success"
)
b2 = widgets.Button(
    description='Watering Problem',
    button_style="warning"
)
b3 = widgets.Button(
    description='Other',
    button_style=""
)
b4 = widgets.Button(
    description='Discard',
    button_style="danger"
)


### Images, process options and buttons

In [22]:
## Get the next image to evaluate
def get_next_image():
    global row
    # Get random row that doesn't have a "category"
    # This will return a list of the values, so we need to
    # remember the order:
    # date;temperature;humidity;moisture
    try:
        row = df[df.category.isna()].sample(1).values[0]
        image = None
        with open(f"photos/{row[0]}.png", "rb") as f:
            image = f.read()
        return image, str(row[1]), str(row[2]), str(row[3])
    except:
        print("Nothing to do. Click on Save!")
        with open(f"plantaris.jpg", "rb") as f:
            image = f.read()
        return image, "", "", ""
    
def process_option(cat):
    if cat == "Ok":
        print("-- Ok")
    elif cat == "Watering Problem":
        print("-- Watering Problem")
    elif cat == "Other":
        print("-- Other")
    elif cat == "Discard":
        print("-- Discard")
    
    # Important
    # Here we will change the value of the dataframe on
    # the category column.
    df.loc[df["date"] == row[0], "category"] = cat
    w_image.value, w_temperature.value, w_humidity.value, w_moisture.value = get_next_image()

# Action for the Buttons
def on_button_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        cat = _.description
        process_option(_.description)
        
def on_save_button_clicked(_):
    clear_output()
    stamp = datetime.now().strftime("%Y_%m_%d-%H:%M:%S")
    filename_bak = f"{filename}_{stamp}"
    print(f"Backing up {filename} into {filename_bak}")
    os.rename(filename, filename_bak)
    df.to_csv(filename, sep=";", index=False, quoting=csv.QUOTE_NONNUMERIC)

# Initial image
w_image.value, w_temperature.value, w_humidity.value, w_moisture.value = get_next_image()    

# Connecting the buttons
b1.on_click(on_button_clicked)
b2.on_click(on_button_clicked)
b3.on_click(on_button_clicked)
b4.on_click(on_button_clicked)
save_button.on_click(on_save_button_clicked)


### Output to evaluate

In [23]:
# Output, and widgets distribution.
def make_boxes():
    vbox1 = widgets.VBox([w_image, out])
    vbox2 = widgets.VBox([widgets.Label('Information'),
                          widgets.VBox([w_temperature, w_humidity, w_moisture]),
                          widgets.HBox([b1, b2, b3, b4]),
                          save_button])
    return vbox1, vbox2
vbox1, vbox2 = make_boxes()
widgets.HBox([vbox1, vbox2])